In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score
!pip install openpyxl
# Load dataset
#df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/book_ratings_cleaned.xlsx")
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/book_ratings_cleaned.csv")

In [ ]:
'''
import torch
print("Is GPU available?", torch.cuda.is_available())
!pip install pymc[jax] --quiet
!pip install jax jaxlib --quiet


def evaluate_predictions(true_ratings, predicted_ratings, threshold=7):
    mae = mean_absolute_error(true_ratings, predicted_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))

    # Convert to binary relevance (1 if rating >= threshold, else 0)
    true_binary = (true_ratings >= threshold).astype(int)
    predicted_binary = (predicted_ratings >= threshold).astype(int)

    precision = precision_score(true_binary, predicted_binary, average='micro')
    recall = recall_score(true_binary, predicted_binary, average='micro')

    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")



import arviz as az
import datetime
from skopt import gp_minimize
from skopt.space import Real


search_space = [
    Real(0.1, 3.0, name="sigma_u"),
    Real(0.1, 3.0, name="sigma_b"),
    Real(0.1, 5.0, name="alpha_mu"),
    Real(0.1, 5.0, name="beta_mu")
]


def objective(params):
    sigma_u_val, sigma_b_val, alpha_mu_val, beta_mu_val = params

    with pm.Model() as model:
        pm.set_data_backend("jax")  # Enables GPU acceleration
        pm.set_compute_backend("jax")


        mu = pm.Gamma("mu", alpha=alpha_mu_val, beta=beta_mu_val)


        user_bias = pm.Normal("user_bias", mu=0, sigma=0.5 / np.sqrt(user_rating_counts + 1), shape=num_users)
        book_bias = pm.Normal("book_bias", mu=0, sigma=0.5 / np.sqrt(book_rating_counts + 1), shape=num_books)


        sigma_u = pm.HalfCauchy("sigma_u", beta=sigma_u_val)
        sigma_b = pm.HalfCauchy("sigma_b", beta=sigma_b_val)

        user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
        book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))


        lambda_rating = pm.math.exp(
            mu +
            user_bias[train_user_ids_small] +
            book_bias[train_book_ids_small] +
            (user_factors[train_user_ids_small] * book_factors[train_book_ids_small]).sum(axis=1)
        )


        ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings_small)

        approx = pm.fit(n=2500, method="advi")
        trace = approx.sample(draws=1000)


    predicted_ratings = np.exp(
        trace.posterior["mu"].mean().item() +
        trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[test_user_ids_small] +
        trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[test_book_ids_small] +
        (trace.posterior["user_factors"].mean(dim=("chain", "draw")).values[test_user_ids_small] *
         trace.posterior["book_factors"].mean(dim=("chain", "draw")).values[test_book_ids_small]).sum(axis=1)
    )


    precision = precision_score((test_ratings_small >= 7).astype(int), (predicted_ratings >= 7).astype(int), average='micro')

    return -precision  # Minimize negative precision (maximize precision)

#bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=15, random_state=42, n_jobs=2)

# best vals
best_sigma_u, best_sigma_b, best_alpha, best_beta = result.x
print(f"Optimal sigma_u: {best_sigma_u}, sigma_b: {best_sigma_b}, alpha_mu: {best_alpha}, beta_mu: {best_beta}")

with pm.Model() as best_model:
    mu = pm.Gamma("mu", alpha=best_alpha, beta=best_beta)
    user_bias = pm.Normal("user_bias", mu=0, sigma=0.5 / np.sqrt(user_rating_counts + 1), shape=num_users)
    book_bias = pm.Normal("book_bias", mu=0, sigma=0.5 / np.sqrt(book_rating_counts + 1), shape=num_books)

    sigma_u = pm.HalfCauchy("sigma_u", beta=best_sigma_u)
    sigma_b = pm.HalfCauchy("sigma_b", beta=best_sigma_b)

    user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
    book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))

    lambda_rating = pm.math.exp(
        mu +
        user_bias[train_user_ids_small] +
        book_bias[train_book_ids_small] +
        (user_factors[train_user_ids_small] * book_factors[train_book_ids_small]).sum(axis=1)
    )

    ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings_small)


    approx = pm.fit(n=5000, method="advi")
    best_trace = approx.sample(draws=1000)

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_filename = f"/content/drive/MyDrive/Colab Notebooks/best_pymc_model_{timestamp}.nc"
az.to_netcdf(best_trace, model_filename)

print(f"Saved best PyMC model to {model_filename}")

'''

'\nimport torch\nprint("Is GPU available?", torch.cuda.is_available())\n!pip install pymc[jax] --quiet\n!pip install jax jaxlib --quiet\n\n\ndef evaluate_predictions(true_ratings, predicted_ratings, threshold=7):\n    mae = mean_absolute_error(true_ratings, predicted_ratings)\n    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))\n\n    # Convert to binary relevance (1 if rating >= threshold, else 0)\n    true_binary = (true_ratings >= threshold).astype(int)\n    predicted_binary = (predicted_ratings >= threshold).astype(int)\n\n    precision = precision_score(true_binary, predicted_binary, average=\'micro\')\n    recall = recall_score(true_binary, predicted_binary, average=\'micro\')\n\n    print(f"MAE: {mae:.4f}")\n    print(f"RMSE: {rmse:.4f}")\n    print(f"Precision: {precision:.4f}")\n    print(f"Recall: {recall:.4f}")\n\n\n\nimport arviz as az\nimport datetime\nfrom skopt import gp_minimize\nfrom skopt.space import Real\n\n# Define search space for Bayesian Opti

In [4]:

!pip install pymc==5.19.1 numpyro jax jaxlib scikit-optimize --quiet

import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import datetime
import torch
from skopt import gp_minimize
from skopt.space import Real
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score
from sklearn.model_selection import train_test_split


print("Is GPU available?", torch.cuda.is_available())


#df = pd.read_excel("book_ratings.xlsx")


df = df[['User-ID', 'ISBN', 'Book-Rating']]
df = df[df['Book-Rating'] > 0]  # Remove zero ratings

# filter out users with too few reviews
user_counts = df['User-ID'].value_counts()
valid_users = user_counts[user_counts >= 2].index
df = df[df['User-ID'].isin(valid_users)]

# encode user-ID and ISBN as categorical indices
df['User-Index'] = df['User-ID'].astype("category").cat.codes
df['Book-Index'] = df['ISBN'].astype("category").cat.codes

# make sure we have a contiguous range for indexing
df['User-Index'] = df['User-Index'].astype("category").cat.codes
df['Book-Index'] = df['Book-Index'].astype("category").cat.codes

# rating counts per user and book
user_rating_counts = df.groupby('User-Index')['Book-Rating'].count().values
book_rating_counts = df.groupby('Book-Index')['Book-Rating'].count().values

### revent division by zero
user_rating_counts[user_rating_counts == 0] = 1
book_rating_counts[book_rating_counts == 0] = 1


train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# convert to np arrays for faster access
train_user_ids = train_df['User-Index'].values
test_user_ids = test_df['User-Index'].values
train_book_ids = train_df['Book-Index'].values
test_book_ids = test_df['Book-Index'].values
train_ratings = train_df['Book-Rating'].values
test_ratings = test_df['Book-Rating'].values

# **Update counts after filtering**
num_users = df['User-Index'].nunique()
num_books = df['Book-Index'].nunique()
print(f"Number of unique users: {num_users}, Number of unique books: {num_books}")

# reducing datasize (hardware limitation)
use_full_dataset = False
latent_dim = 3 ### hardware limitation

if use_full_dataset:
    train_user_ids_small, train_book_ids_small, train_ratings_small = train_user_ids, train_book_ids, train_ratings
    test_user_ids_small, test_book_ids_small, test_ratings_small = test_user_ids, test_book_ids, test_ratings
else:
    subset_fraction = 0.01  # 1% of training data
    subset_size = int(len(train_df) * subset_fraction)
    subset_size = max(1000, min(subset_size, 20000))

    train_idx_subset = np.random.choice(len(train_user_ids), subset_size, replace=False)
    test_idx_subset = np.random.choice(len(test_user_ids), min(subset_size // 2, len(test_user_ids)), replace=False)

    train_user_ids_small, train_book_ids_small, train_ratings_small = (
        train_user_ids[train_idx_subset],
        train_book_ids[train_idx_subset],
        train_ratings[train_idx_subset],
    )

    test_user_ids_small, test_book_ids_small, test_ratings_small = (
        test_user_ids[test_idx_subset],
        test_book_ids[test_idx_subset],
        test_ratings[test_idx_subset],
    )

    print(f"Using {subset_size} training rows and {len(test_user_ids_small)} test rows.")

#### evaluate our preds
def evaluate_predictions(true_ratings, predicted_ratings, threshold=7):
    mae = mean_absolute_error(true_ratings, predicted_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    true_binary = (true_ratings >= threshold).astype(int)
    predicted_binary = (predicted_ratings >= threshold).astype(int)
    precision = precision_score(true_binary, predicted_binary, average='micro')
    recall = recall_score(true_binary, predicted_binary, average='micro')
    print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

# limited bayesian optimization search space --> limited by hardware
search_space = [
    Real(0.1, 3.0, name="sigma_u"),
    Real(0.1, 3.0, name="sigma_b"),
    Real(0.1, 5.0, name="alpha_mu"),
    Real(0.1, 5.0, name="beta_mu"),
]

# objective function for optimization
def objective(params):
    sigma_u_val, sigma_b_val, alpha_mu_val, beta_mu_val = params
    with pm.Model() as model:
        mu = pm.Gamma("mu", alpha=alpha_mu_val, beta=beta_mu_val)
        user_bias = pm.Normal("user_bias", mu=0, sigma=0.5 / np.sqrt(user_rating_counts + 1), shape=num_users)
        book_bias = pm.Normal("book_bias", mu=0, sigma=0.5 / np.sqrt(book_rating_counts + 1), shape=num_books)
        sigma_u = pm.HalfCauchy("sigma_u", beta=sigma_u_val)
        sigma_b = pm.HalfCauchy("sigma_b", beta=sigma_b_val)
        user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
        book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))
        lambda_rating = pm.math.exp(mu + user_bias[train_user_ids_small] + book_bias[train_book_ids_small] +
                                    (user_factors[train_user_ids_small] * book_factors[train_book_ids_small]).sum(axis=1))
        ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings_small)
        approx = pm.fit(n=2500, method="advi")
        trace = approx.sample(draws=500)  # reduced draws to minimize RAM usage --- hardware limitation

    predicted_ratings = np.exp(trace.posterior["mu"].mean().item() +
                               trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[test_user_ids_small] +
                               trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[test_book_ids_small] +
                               (trace.posterior["user_factors"].mean(dim=("chain", "draw")).values[test_user_ids_small] *
                                trace.posterior["book_factors"].mean(dim=("chain", "draw")).values[test_book_ids_small]).sum(axis=1))

    precision = precision_score((test_ratings_small >= 7).astype(int), (predicted_ratings >= 7).astype(int), average='micro')
    return -precision




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.9/501.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.8/360.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.9 MB/s eta 0:00:00
Is GPU available? True


<ipython-input-4-aab5ff30f8ab>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User-Index'] = df['User-ID'].astype("category").cat.codes


Number of unique users: 28868, Number of unique books: 141472
Using 2756 training rows and 1378 test rows.


In [ ]:
# **Run Bayesian Optimization**
#result = gp_minimize(objective, search_space, n_calls=10, random_state=42, n_jobs=1)

#print(f"Optimal parameters: {result.x}")


In [5]:
# retraining final model

best_sigma_u = 1.2595968179742412
best_sigma_b = 0.23533042331948476
best_alpha = 4.871402042323151
best_beta = 1.2405795681084912

with pm.Model() as best_model:
    mu = pm.Gamma("mu", alpha=best_alpha, beta=best_beta)
    user_bias = pm.Normal("user_bias", mu=0, sigma=0.5 / np.sqrt(user_rating_counts + 1), shape=num_users)
    book_bias = pm.Normal("book_bias", mu=0, sigma=0.5 / np.sqrt(book_rating_counts + 1), shape=num_books)

    sigma_u = pm.HalfCauchy("sigma_u", beta=best_sigma_u)
    sigma_b = pm.HalfCauchy("sigma_b", beta=best_sigma_b)

    user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
    book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))

    lambda_rating = pm.math.exp(
        mu + user_bias[train_user_ids_small] + book_bias[train_book_ids_small] +
        (user_factors[train_user_ids_small] * book_factors[train_book_ids_small]).sum(axis=1)
    )

    ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings_small)

    # using jax to accelarate the sampling (numpyro) and utilize multiprocessing
    best_trace = pm.sample(
        draws=500, tune=500, chains=2,
        nuts_sampler="numpyro",
        nuts_sampler_kwargs={"chain_method": "vectorized"}
    )


timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_filename = f"best_pymc_model_{timestamp}.nc"
az.to_netcdf(best_trace, model_filename)

print(f"Saved best PyMC model to {model_filename}")


sample: 100%|██████████| 1000/1000 [48:29<00:00,  2.91s/it]
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Saved best PyMC model to best_pymc_model_2025-03-08_05-38-46.nc


In [6]:
model_filename = f"/content/drive/MyDrive/Colab Notebooks/best_pymc_model_{timestamp}.nc"
az.to_netcdf(best_trace, model_filename)

'/content/drive/MyDrive/Colab Notebooks/best_pymc_model_2025-03-08_05-38-46.nc'

In [7]:

import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"/content/drive/MyDrive/beta_alpha_sigma_{timestamp}.txt"
with open(filename, "w") as f:
    f.write(f"Optimal sigma_u: {best_sigma_u}\n")
    f.write(f"Optimal sigma_b: {best_sigma_b}\n")
    f.write(f"Optimal alpha_mu: {best_alpha}\n")
    f.write(f"Optimal beta_mu: {best_beta}\n")

print(f"Saved parameters to {filename}")


Saved parameters to /content/drive/MyDrive/beta_alpha_sigma_2025-03-08_05-47-40.txt


In [ ]:


best_sigma_u, best_sigma_b, best_alpha, best_beta = result.x
print(f"Optimal sigma_u: {best_sigma_u}, sigma_b: {best_sigma_b}, alpha_mu: {best_alpha}, beta_mu: {best_beta}")

